# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
EPS = 1e-15

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_graph(nodes_number):
    edges = pd.read_csv('datasets/email.csv', header=None).iloc[1:,]
    edges = edges.astype('int')
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[0]):
        graph_np[edges.iloc[i, 0], edges.iloc[i, 1]]=1
        graph_np[edges.iloc[i, 1], edges.iloc[i, 0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.5),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.5)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
nodes_number = 986
train_edge, test_edge, train_mask, test_mask = get_graph(nodes_number)

In [6]:
embedding_dim = 12
n_clusters = 24
alpha = 1
beta = 4.5
DEC = 5.0
LR = 0.1

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.7631006103390997
AUC SCORE:  0.7533264368447525
AP：  0.7960736652373135
AUC SCORE:  0.7904087320414557
######################### Module cycle ： 1 ##########################
AP：  0.8108170903598492
AUC SCORE:  0.7994484493549705
AP：  0.8104016751658047
AUC SCORE:  0.7992734536827788
######################### Module cycle ： 2 ##########################
AP：  0.8255484171627674
AUC SCORE:  0.8168723186809534
AP：  0.8288683145236468
AUC SCORE:  0.8220691917960626
######################### Module cycle ： 3 ##########################
AP：  0.8357502946340049
AUC SCORE:  0.829539187312007
AP：  0.8380770391578183
AUC SCORE:  0.8323289650784312
######################### Module cycle ： 4 ##########################
AP：  0.8430558950195131
AUC SCORE:  0.8372332306109228
AP：  0.8451941731741645
AUC SCORE:  0.8393092067735035
######################### Module cycle ： 5 ##########################
AP：  0.8500417798260718
AUC SCO

AP：  0.9136372771111277
AUC SCORE:  0.9111434537621427
######################### Module cycle ： 46 ##########################
AP：  0.9137559641634001
AUC SCORE:  0.9113255797775035
AP：  0.9138303183345599
AUC SCORE:  0.911424846555015
######################### Module cycle ： 47 ##########################
AP：  0.9139307255103908
AUC SCORE:  0.9115996639686275
AP：  0.9140557326467152
AUC SCORE:  0.9117856186682554
######################### Module cycle ： 48 ##########################
AP：  0.9141378246786266
AUC SCORE:  0.9119149878945374
AP：  0.9143281841813041
AUC SCORE:  0.9121629610789909
######################### Module cycle ： 49 ##########################
AP：  0.9143513961426492
AUC SCORE:  0.9122325129263822
AP：  0.9144265714843899
AUC SCORE:  0.9123533102401151
######################### Module cycle ： 50 ##########################
AP：  0.9144795911058325
AUC SCORE:  0.9124310542317782
AP：  0.9145195718744565
AUC SCORE:  0.9125128981707632
######################### Module cycle ： 

AP：  0.915969095360389
AUC SCORE:  0.9167793777578928
AP：  0.915980253599504
AUC SCORE:  0.9168252289646195
######################### Module cycle ： 92 ##########################
AP：  0.9159711415085792
AUC SCORE:  0.91683344436001
AP：  0.9159890985344746
AUC SCORE:  0.9168824034663141
######################### Module cycle ： 93 ##########################
AP：  0.9159767711366211
AUC SCORE:  0.9168943932933604
AP：  0.9159903643606331
AUC SCORE:  0.9169419573325225
######################### Module cycle ： 94 ##########################
AP：  0.9159796455284431
AUC SCORE:  0.9169528543569745
AP：  0.9159987921191388
AUC SCORE:  0.9170025342479683
######################### Module cycle ： 95 ##########################
AP：  0.9159907628234705
AUC SCORE:  0.9170150898522443
AP：  0.9160079740205198
AUC SCORE:  0.9170607860515111
######################### Module cycle ： 96 ##########################
AP：  0.9159964265731841
AUC SCORE:  0.9170739771863741
AP：  0.9160158669989563
AUC SCORE:  0.917123

AP：  0.9163655684208855
AUC SCORE:  0.9187143668850495
######################### Module cycle ： 137 ##########################
AP：  0.91635241030307
AUC SCORE:  0.9187069187765868
AP：  0.9163681763899059
AUC SCORE:  0.9187411444237988
######################### Module cycle ： 138 ##########################
AP：  0.9163520713478867
AUC SCORE:  0.9187340605828677
AP：  0.9163651924783919
AUC SCORE:  0.9187636747581388
######################### Module cycle ： 139 ##########################
AP：  0.9163516212004913
AUC SCORE:  0.9187556453717004
AP：  0.916366330771119
AUC SCORE:  0.9187855850626379
######################### Module cycle ： 140 ##########################
AP：  0.916352741090493
AUC SCORE:  0.9187756878363041
AP：  0.9163620273450327
AUC SCORE:  0.9188002952706108
######################### Module cycle ： 141 ##########################
AP：  0.9163499218413121
AUC SCORE:  0.918795032767337
AP：  0.9163603658864875
AUC SCORE:  0.9188213530340787
######################### Module cycle ：

AP：  0.915916278788991
AUC SCORE:  0.9191978196526643
AP：  0.91591642143604
AUC SCORE:  0.9192080036427994
######################### Module cycle ： 183 ##########################
AP：  0.9159129152881031
AUC SCORE:  0.919207120100276
AP：  0.9159137136955162
AUC SCORE:  0.9192185441500929
######################### Module cycle ： 184 ##########################
AP：  0.9159065630808099
AUC SCORE:  0.9192175986045856
AP：  0.9159069053853989
AUC SCORE:  0.9192288676469421
######################### Module cycle ： 185 ##########################
AP：  0.9159022019412719
AUC SCORE:  0.9192301929607268
AP：  0.9159028994296936
AUC SCORE:  0.9192402064426558
######################### Module cycle ： 186 ##########################
AP：  0.9158934433031767
AUC SCORE:  0.9192398344247512
AP：  0.9158921441091544
AUC SCORE:  0.919248166075737
######################### Module cycle ： 187 ##########################
AP：  0.9158821532481838
AUC SCORE:  0.9192447094093744
AP：  0.9158851703337174
AUC SCORE:  0.91

...

AP：  0.9160048146829776
AUC SCORE:  0.915145754134855
AP：  0.9160097483929606
AUC SCORE:  0.9151796310152815
######################### Module cycle ： 83 ##########################
AP：  0.915987381586306
AUC SCORE:  0.9152095087032348
AP：  0.9159927805830262
AUC SCORE:  0.9152451914205726
######################### Module cycle ： 84 ##########################
AP：  0.9159723927218517
AUC SCORE:  0.9152736197887732
AP：  0.9159830046455593
AUC SCORE:  0.9153071556527872
######################### Module cycle ： 85 ##########################
AP：  0.9159627807501911
AUC SCORE:  0.9153341734531
AP：  0.9159686245847036
AUC SCORE:  0.9153675775607729
######################### Module cycle ： 86 ##########################
AP：  0.9159482346913652
AUC SCORE:  0.915394920876752
AP：  0.9159603759941974
AUC SCORE:  0.9154299138108919
######################### Module cycle ： 87 ##########################
AP：  0.9159479343958568
AUC SCORE:  0.9154583266783464
AP：  0.9159571949141729
AUC SCORE:  0.91549133

AP：  0.9168320542156889
AUC SCORE:  0.9182511503413636
######################### Module cycle ： 128 ##########################
AP：  0.9168452254148808
AUC SCORE:  0.9182855542471549
AP：  0.9168451358861234
AUC SCORE:  0.9183010394924287
######################### Module cycle ： 129 ##########################
AP：  0.9168568300449778
AUC SCORE:  0.9183300258874859
AP：  0.9168571140961299
AUC SCORE:  0.9183420622167703
######################### Module cycle ： 130 ##########################
AP：  0.9168659003018275
AUC SCORE:  0.9183690567659641
AP：  0.9168645641219286
AUC SCORE:  0.9183795352702735
######################### Module cycle ： 131 ##########################
AP：  0.9168798320240331
AUC SCORE:  0.918410955282456
AP：  0.916882461508096
AUC SCORE:  0.918421991813622
######################### Module cycle ： 132 ##########################
AP：  0.9168967925913263
AUC SCORE:  0.9184553184175648
AP：  0.9168980827127553
AUC SCORE:  0.9184661689397788
######################### Module cycle

AP：  0.9173195611658839
AUC SCORE:  0.9197275808990135
AP：  0.9173201708959834
AUC SCORE:  0.9197366178339431
######################### Module cycle ： 174 ##########################
AP：  0.9173249683113073
AUC SCORE:  0.9197495222050046
AP：  0.917325841202637
AUC SCORE:  0.9197589621593307
######################### Module cycle ： 175 ##########################
AP：  0.9173253700992826
AUC SCORE:  0.9197705567173537
AP：  0.9173298039457726
AUC SCORE:  0.9197819032634403
######################### Module cycle ： 176 ##########################
AP：  0.9173301240264883
AUC SCORE:  0.9197937923356375
AP：  0.9173314831305476
AUC SCORE:  0.9198038678205505
######################### Module cycle ： 177 ##########################
AP：  0.9173358590043279
AUC SCORE:  0.9198164234248266
AP：  0.917335402959901
AUC SCORE:  0.9198256851205735
######################### Module cycle ： 178 ##########################
AP：  0.9173364580094572
AUC SCORE:  0.9198348615622172
AP：  0.91734069845675
AUC SCORE:  0.9

AP：  0.9172689601746071
AUC SCORE:  0.9205082062189489
######################### Module cycle ： 219 ##########################
AP：  0.9172630861910075
AUC SCORE:  0.9205204053060665
AP：  0.9172549756664615
AUC SCORE:  0.920519963534805
######################### Module cycle ： 220 ##########################
AP：  0.917262247511327
AUC SCORE:  0.9205342242211432
AP：  0.9172539075451195
AUC SCORE:  0.9205347124946429
######################### Module cycle ： 221 ##########################
AP：  0.9172565971080615
AUC SCORE:  0.920549608711568
AP：  0.9172521700311171
AUC SCORE:  0.9205481516414422
######################### Module cycle ： 222 ##########################
AP：  0.9172544889961228
AUC SCORE:  0.920562071311368
AP：  0.9172458463959614
AUC SCORE:  0.9205602499737109
######################### Module cycle ： 223 ##########################
AP：  0.9172481115325986
AUC SCORE:  0.9205762079917382
AP：  0.9172377411471154
AUC SCORE:  0.920571379509353
######################### Module cycle ：

AP：  0.9173713871033602
AUC SCORE:  0.9212373148094872
AP：  0.9173736642666358
AUC SCORE:  0.9212486846066927
######################### Module cycle ： 265 ##########################
AP：  0.9173728285330428
AUC SCORE:  0.92125042844062
AP：  0.9173764579691056
AUC SCORE:  0.9212616587311114
######################### Module cycle ： 266 ##########################
AP：  0.9173718818681225
AUC SCORE:  0.9212630925501182
AP：  0.9173747303676683
AUC SCORE:  0.9212747491111253
######################### Module cycle ： 267 ##########################
AP：  0.9173731654251136
AUC SCORE:  0.9212756559047673
AP：  0.9173774611072113
AUC SCORE:  0.9212888625403763
######################### Module cycle ： 268 ##########################
AP：  0.9173781425972283
AUC SCORE:  0.9212884982728449
AP：  0.917378150392941
AUC SCORE:  0.9212980467323931
######################### Module cycle ： 269 ##########################
AP：  0.9173772383152119
AUC SCORE:  0.9212998060670664
AP：  0.9173779684965017
AUC SCORE:  0.

In [8]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.9177582067812139
AP FIRST AP STD:  0.001208861191715632
AP FIRST AUC MEAN:  0.9211114913788571
AP FIRST AUC STD:  0.001835993333912895
AUC FIRST AP MEAN:  0.9169251101640583
AUC FIRST AP STD:  0.001462719734410829
AUC FIRST AUC MEAN:  0.9224530111935227
AUC FIRST AUC STD:  0.0013576640519242394
